In [1]:
import psycopg2
# import sqlalchemy as sq
import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [2]:
# conn_lims = pyodbc.connect("Driver={SQL Server};"
#                             "Server=192.168.5.18\CROPNUT;"
#                             "Database=cropnuts;"
#                             "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [3]:
# _ = pd.read_sql("select TOP(1000) * from SampleResults where group_name='Soil Analysis' AND analysis_name LIKE '%Complete Soil Analysis%'",con=conn_lims)

In [4]:
# _.to_csv("sl.csv")

In [26]:
soil_df = pd.read_csv("input/soil_analysis_cleaned.csv")

In [27]:
len(soil_df)

3680915

In [28]:
soil_df['analysis_name'] = soil_df['analysis_name'].str.split(",")

In [29]:
soil_df = soil_df.explode('analysis_name', ignore_index=True)


In [30]:
soil_df.to_csv("output/soil_single_analysis.csv")

In [31]:
soil_df = pd.read_csv("output/soil_single_analysis.csv")

In [32]:
soil_df.chemical_name.unique()

array(['iron', 'cec', 'aluminium', 'ec_salts', 'organic_matter',
       'phosphorus', 'potassium', 'calcium', 'magnesium', 'manganese',
       'sulphur', 'copper', 'boron', 'zinc', 'sodium', 'phosphorus_olsen',
       'exchangeable_acidity', 'phosphorus_sorption_index_psi',
       'cn_ratio', 'acid_saturation', 'nitrogen',
       'exchangeable_aluminium', 'clay', 'organic_carbon',
       'total_nitrogen', 'ph', 'silt', 'sand', 'nitrate_n', 'ammonium',
       'nitrates', 'bulk_density', 'silicon__', 'cobalt', 'ph_kcl',
       'caco3', 'arsenic', 'cadmium', 'chromium', 'mercury', 'nickel',
       'lead', 'electrical_conductivity', 'cec_nh4oac', 'molybdenum',
       'phosphorus_bray2', 'co2', 'microbial_activity_co2_respiration',
       'total_carbon', 'total_phosphorus', 'total_organic_carbon',
       'soil_life_co2_burst', 'soil_moisture_', '%_gypsum',
       'phosphorous_p1', 'free_lime', 'e_coli', 'paramagnetism',
       'morgan-k', 'morgan-ca', 'morgan-mg', 'morgan-s', 'morgan-no3',


In [33]:
soil_df = soil_df.dropna(subset="result")

In [34]:
df_ = soil_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/soil_unit_per_chemical.csv")

In [35]:
df_ = df_.reset_index()

In [36]:
df_.chemical_name.unique()

array(['zinc', 'cec', 'magnesium', 'calcium', 'manganese', 'iron',
       'potassium', 'sodium', 'total_nitrogen', 'sulphur',
       'organic_matter', 'copper', 'boron', 'ec_salts', 'phosphorus',
       'organic_carbon', 'aluminium', 'clay', 'sand', 'silt', 'nitrogen',
       'exchangeable_acidity', 'phosphorus_sorption_index_psi',
       'phosphorus_olsen', 'nitrate_n', 'total_carbon', 'ammonium',
       'exchangeable_aluminium', 'silicon__', 'cn_ratio', 'edta_iron',
       'edta_zinc', 'edta_manganese', 'sulphate', 'edta_copper',
       'acid_saturation', 'reactive_carbon', 'cobalt',
       'total_organic_carbon', 'ammoniacal_n', 'bulk_density',
       'total_phosphorus', 'ph', 'nickel', 'arsenic', 'chromium',
       'cadmium', 'lead', 'nitrates', 'calcium_carbonate', 'molybdenum',
       'phosphorus_bray2', 'morgan-k', 'morgan-s', 'morgan-mg',
       'morgan-p', 'morgan-ca', 'morgan-nh4', 'morgan-no3',
       'soil_moisture_', 'phosphorus_p1', 'ph_kcl', 'exch_acidity_kcl',
       'e

In [40]:
analysis_chem_unit_df = pd.DataFrame()
for analysis in df_['analysis_name'].unique():
    print(analysis)
    chems_count_ = df_.loc[df_['analysis_name'] == analysis]
    duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
    temp_analysis_chem_unit_df = pd.DataFrame()
    for index,row in chems_count_.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"analysis_name":[analysis],"chemical_name":[chemical], "unit_name":[unit]})
        temp_analysis_chem_unit_df = pd.concat([temp_analysis_chem_unit_df, _])
    analysis_chem_unit_df = pd.concat([analysis_chem_unit_df, temp_analysis_chem_unit_df])
analysis_chem_unit_df.to_csv("output/soil_unit_per_chemical_decision.csv")

Complete Soil Analysis with Recommendations
Basic Soil Analysis with Recommendations
Complete Soil Analysis
Soil Texture Analysis
Complete Soil Analysis (M3)
AFSIS Std Wet Chemistry Soil Analysis
Soil Carbon (Walkley Black)
Complete Soil Analysis (data only)
Standard Wet Chem Soil Analysis  + Micros
Basic Soil Analysis
Spectral Soil Texture Analysis
Available Nitrogen in Soil
Sulphur Analysis
Soil Nitrogen (Kjeldahl)
Standard Soil (Olsen P) + Micros
Total Carbon & Nitrogen
% Organic Matter (OM)
% Soil Nitrogen (N)
Cropnuts Wet Chem IR Calibration Soil Analysis
Available Nitrogen
Exchangeable Acidity (Hp) Analysis
ATA Ethiopia Soil Analysis
Soil Organic Carbon and Total Nitrogen (IR)
 Mn)
 Fe
Zn
Soil Micronutrients (EDTA Cu
Soil sulphate (calcium phosphate)
KTDA Soil Analysis
Exchangeable Aluminium in Soil
Complete Soil Analysis + Exch. Acidity
Mehlich 3 extract analysis
Soil pH in Water
Complete Soil Analysis - Co & Si (IFDC)
Exchangeable Acidity (Hp) in Soil
Standard Wet Chem Soil Ana

In [41]:
analysis_chem_unit_df

,analysis_name,chemical_name,unit_name
0,Complete Soil Analysis with Recommendations,zinc,ppm
0,Complete Soil Analysis with Recommendations,cec,meq/100g
0,Complete Soil Analysis with Recommendations,magnesium,ppm
0,Complete Soil Analysis with Recommendations,calcium,ppm
0,Complete Soil Analysis with Recommendations,manganese,ppm
...,...,...,...
0,Phosphorus Analysis,magnesium,ppm
0,Phosphorus Analysis,ec_salts,uS/cm
0,Phosphorus Analysis,cec,meq/100g
0,Phosphorus Analysis,calcium,ppm


In [42]:
analysis_chem_unit_df.chemical_name.unique()

array(['zinc', 'cec', 'magnesium', 'calcium', 'manganese', 'iron',
       'potassium', 'sodium', 'total_nitrogen', 'sulphur',
       'organic_matter', 'copper', 'boron', 'ec_salts', 'phosphorus',
       'organic_carbon', 'phosphorus_olsen', 'clay', 'sand', 'silt',
       'cn_ratio', 'acid_saturation', 'exchangeable_acidity',
       'exchangeable_aluminium', 'nitrate_n', 'reactive_carbon', 'ph',
       'aluminium', 'ammoniacal_n', 'cobalt', 'nickel', 'arsenic',
       'cadmium', 'chromium', 'lead', 'molybdenum', 'calcium_carbonate',
       'ammonium', 'est_available_water_holding_capacity',
       'soil_life_co2_burst', '_total_nitrogen',
       'water_stable_aggregate_', 'phosphorus_p1', 'nitrates',
       'microbial_activity_co2_respiration', 'bulk_density',
       'total_carbon', 'water_holding_capacity', 'morgan-p', 'chloride',
       'microbial_co2_burst', 'exch_acidity_kcl', 'soil_moisture_',
       'morgan-mg', 'morgan-ca', 'morgan-k', 'gold', 'e_coli',
       'mineral_carbon', '

In [43]:
df = pd.DataFrame()
for index, row in analysis_chem_unit_df.iterrows():
    analysis = row['analysis_name']
    chemical = row['chemical_name']
    unit = row['unit_name']
    if chemical == "phosphorus_olsen":
        print(analysis)
        print(chemical)
        print(unit)
    temp_df = soil_df.loc[(soil_df['analysis_name'] == analysis) & (soil_df['chemical_name'] == chemical) & (soil_df['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

Complete Soil Analysis with Recommendations
phosphorus_olsen
ppm
Basic Soil Analysis with Recommendations
phosphorus_olsen
ppm
Complete Soil Analysis
phosphorus_olsen
ppm
Soil Texture Analysis
phosphorus_olsen
ppm
Complete Soil Analysis (M3)
phosphorus_olsen
ppm
Soil Carbon (Walkley Black)
phosphorus_olsen
ppm
Complete Soil Analysis (data only)
phosphorus_olsen
ppm
Standard Wet Chem Soil Analysis  + Micros
phosphorus_olsen
ppm
Basic Soil Analysis
phosphorus_olsen
ppm
Spectral Soil Texture Analysis
phosphorus_olsen
ppm
Available Nitrogen in Soil
phosphorus_olsen
ppm
Soil Nitrogen (Kjeldahl)
phosphorus_olsen
ppm
Standard Soil (Olsen P) + Micros
phosphorus_olsen
ppm
Total Carbon & Nitrogen
phosphorus_olsen
ppm
% Organic Matter (OM)
phosphorus_olsen
ppm
% Soil Nitrogen (N)
phosphorus_olsen
ppm
Available Nitrogen
phosphorus_olsen
ppm
Exchangeable Acidity (Hp) Analysis
phosphorus_olsen
ppm
Exchangeable Aluminium in Soil
phosphorus_olsen
ppm
Complete Soil Analysis + Exch. Acidity
phosphorus_o

In [44]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
5097,5097,3170,3170,FA807-384SA0001,2019-10-30 00:00:00.000,Complete Soil Analysis with Recommendations,zinc,1.33,ppm
5430,5430,3315,3315,CH006SA3543,2019-05-17 00:00:00.000,Complete Soil Analysis with Recommendations,zinc,15.30,ppm
5592,5592,3405,3405,CH006SA3544,2019-05-17 00:00:00.000,Complete Soil Analysis with Recommendations,zinc,19.40,ppm
5626,5626,3419,3419,FA611-161SA0001,2019-04-29 00:00:00.000,Complete Soil Analysis with Recommendations,zinc,3.18,ppm
6114,6114,3792,3792,CH006SA3545,2019-05-17 00:00:00.000,Complete Soil Analysis with Recommendations,zinc,11.30,ppm
...,...,...,...,...,...,...,...,...,...
1304356,1304356,925417,925417,CI016SA1311,2014-06-10 00:00:00.000,Phosphorus Analysis,magnesium,58.30,ppm
692036,692036,455924,455924,CI016SA1311,2014-06-10 00:00:00.000,Phosphorus Analysis,ec_salts,127.00,uS/cm
692041,692041,455925,455925,CI016SA1311,2014-06-10 00:00:00.000,Phosphorus Analysis,cec,7.40,meq/100g
1304351,1304351,925416,925416,CI016SA1311,2014-06-10 00:00:00.000,Phosphorus Analysis,calcium,265.00,ppm


In [45]:
df.to_csv("output/soil_analysis_cleaned.csv",chunksize=1000)

In [26]:
os.getcwd()

'D:\\Cropnuts\\DSML158\\SoilAnalysis'